# Insula API Data Access & Discovery

## Overview

This document serves as a quick guide of how to access Insula's API for various tasks in Python. This guide will show how to discover, filter and download data.

***Steps***:
1. Preparation
2. Catalog Search
3. Product Search and Download

## Preparation

These preparation blocks have to be executed as a prerequisite for every subsequent step to work correctly, as these variables will be used to configure each http request.
Be sure to put a valid `USERNAME/TOKEN` pair, and check that the domain inside `BASE_URL` is still valid.

In [ ]:
import requests
import base64
import json
import getpass

In [ ]:
USERNAME=getpass.getpass()

In [ ]:
PASSWORD=getpass.getpass()

In [ ]:
from InsulaWorkflowClient import InsulaOpenIDConnect

BASE_URL="https://insula.earth"

insulaAuth: InsulaOpenIDConnect = InsulaOpenIDConnect(
        authorization_endpoint="https://identity.ope.insula.earth/realms/eopaas/protocol/openid-connect/auth",
        token_endpoint="https://identity.ope.insula.earth/realms/eopaas/protocol/openid-connect/token",
        redirect_uri="http://localhost:9207/auth",
        client_id="api-client"
    )
insulaAuth.set_user_credentials(username=USERNAME, password=PASSWORD)

bearer = insulaAuth.get_authorization_header()
print(bearer)
HEADERS={'Authorization': bearer }

## Product Search and Download

### Get Collections Info

This command could be run to find the available collections inside the platform and should be pretty straightforward.

In [ ]:
url=BASE_URL+"/secure/api/v2.0/collections?size=20"
run_request=requests.get(url,headers=HEADERS, verify=True)

run_request_dict = json.loads(run_request.text)

In [ ]:
run_request_dict

### Get File Info from Collection

This request returns us a list of files inside a particular collection. We can get a valid `collection_id` with the help of the previous block or put a known one.

In [ ]:
collection_id=52
my_collection=f'{BASE_URL}/secure/api/v2.0/collections/{collection_id}'
url=f"{BASE_URL}/secure/api/v2.0/platformFiles/search/parametricFind?sort=filename&type=OUTPUT_PRODUCT&collection={my_collection}"
run_request=requests.get(url,headers=HEADERS, verify=True)
run_request_dict = json.loads(run_request.text)

In [ ]:
run_request_dict

### Download file

We're getting a valid `file_id` from the request done in the last section. A custom one can be placed here if needed

In [ ]:
file_id=run_request_dict['_embedded']['platformFiles'][0]['id']
print(f"file_id: {file_id}")

In this section the file download is done through the help of a function to which we have to provide the following parameters:
- `url`: the path to a specific file inside the platform, this contains the `file_id`.
- `local_filename`: the name of the downloaded file
- `customHeader`: through this we can pass the `HEADERS` containing our auth, as the request is done inside the function.

In [ ]:
def download_file(url, local_filename, customHeader, checkCert=True):
    print(f'{url} started')
    with requests.get(url,headers=customHeader, stream=True, verify=checkCert ) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):  
                f.write(chunk)

    print(f'download {url} done')
    return local_filename

In [ ]:
file_to_download = BASE_URL + "/secure/api/v2.0/platformFiles/" + str(file_id)+"/dl"
download_file(file_to_download,'downloadTestFile.tif',HEADERS)